Ada López del Castillo Avilés, NIU:1605347

### Exercici 1: Càlcul manual de Gini, entropia i error rate

Considera el següent node d'un arbre de classificació binària:

A la regió hi ha 20 observacions (15 classe 1 (positiva) i 5 de classe 0 (negativa)).
- Calcula les probabilitats empíriques de cada classe al node.
- Calcula l'error rate del node.
- Calcula l'índex de Gini del node.
- Calcula l'entropia (en bits, base 2) del node.
Interpreta els valors obtinguts: el node és molt pur, una mica impur...?
Després, suposa que fem un split d'aquest node en dos fills amb les següents distribucions:

Fill esquerre: 8 positius, 2 negatius (10 observacions en total).
Fill dret: 7 positius, 3 negatius (10 observacions en total).
- Calcula les probabilitats, Gini, entropia i error rate de cadascun dels fills.
- Calcula la impuresa ponderada (pel Gini i per l'entropia) del split.
- Compara la impuresa abans i després del split: ha millorat la puresa? En quin sentit?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    ConfusionMatrixDisplay
)

# Perquè les figures surtin dins del notebook
%matplotlib inline

# Fixem llavor per a la reproductibilitat
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

def entropy(p):
    """Entropia de Shannon per a un vector de probabilitats p (base 2)."""
    p = np.asarray(p, dtype=float)
    p = p[p > 0]  # eliminem probabilitats 0 per evitar log(0)
    return -np.sum(p * np.log2(p))

def gini(p):
    """Índex de Gini per a un vector de probabilitats p."""
    p = np.asarray(p, dtype=float)
    return 1.0 - np.sum(p**2)

def error_rate(p):
    """Error rate = 1 - max_k p_k."""
    p = np.asarray(p, dtype=float)
    return 1.0 - np.max(p)

In [ ]:
# Construïm el dataset amb les 20 observacions
data_ex1 = pd.DataFrame({
    "Y":  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
})

data_ex1

In [ ]:
#Mirem quants hi ha de cada classe
counts_root = data_ex1['Y'].value_counts().sort_index()
counts_root

In [ ]:
# Probabilitats empíriques de cada classe al node arrel
p_root = counts_root / counts_root.sum()

# Calculem les probabilitats empíriques, l'error rate, l'índex de Gini i l'entropia del node
print("Probabilitats al node arrel:", p_root.to_dict())
print("Error rate:", error_rate(p_root))
print("Gini:", gini(p_root))
print("Entropia (bits):", entropy(p_root))

In [ ]:
#Ara fem un split d'aquest node en dos fills:

p_left  = np.array([2/10, 8/10])   # fill esquerre
p_right = np.array([3/10, 7/10])   # fill dret

print("Fill esquerre (8 positius, 2 negatius)")
print("Probabilitats:", p_left)
print("Error rate:", error_rate(p_left))
print("Gini:", gini(p_left))
print("Entropia:", entropy(p_left))
print()

print("Fill dret (7 positius, 3 negatius)")
print("Probabilitats:", p_right)
print("Error rate:", error_rate(p_right))
print("Gini:", gini(p_right))
print("Entropia:", entropy(p_right))
print()

#Impuresa ponderada del split
w_left = 10/20
w_right = 10/20

Gini_split = w_left * gini(p_left) + w_right * gini(p_right)
Entropia_split = w_left * entropy(p_left) + w_right * entropy(p_right)
print("Gini amb impuresa ponderada:", Gini_split)
print("Entropia amb impuresa ponderada:", Entropia_split)
print()

#Imprimim les dades abans i després del split
print("Gini abans del split:", gini(p_root))
print("Gini després del split:", Gini_split)
print("Entropia abans del split:", entropy(p_root))
print("Entropia després del split:", Entropia_split)

S'ha construït un dataframe amb 20 observacions, 15 de classe 1 i 5 de classe 0 (positiva i negativa respectivament), per això les probabilitats empíriques són p(1)=0.75 i p(0)= 0.25.
Després s'ha calculat l'error rate i n'ha sortit 0.25, indicant que la regla de classificació que assigna sempre la classe majoritària (la positiva) s'equivocaria en un 25% dels casos.

L'índex de Gini val 0.375 i l'entropia 0.81 bits indicant que el node no és pur (pur seria valors 0), però donat que hi ha una classe clarament dominant, la impuresa no és màxima.  

Ara, després de fer el split notem que els valors de Gini i entropia disminueixen molt lleugerament, això vol dir que disminueix la impuresa, ja que a menys valor, més pur, però d'una manera molt lleugera, per tant, els nodes fills són una mica més purs que el node original.

### Exercici 2: Dades simulades 2D i sobreajustament (overfitting)

Genera un conjunt de dades 2D de classificació binària amb la funció make_blobs de sklearn.datasets, amb:
- 2 features (coordenades en 2D).
- 2 centres ben separats.
- 500 observacions.
  
Fes un train/test split amb test_size=0.3 i random_state=RANDOM_STATE.

Entrena dos arbres de decisió:
- Arbre A: max_depth=None (sense límit, arbre molt profund).
- Arbre B: max_depth=3.

Calcula l'accuracy de cadascun dels models en train i en test.

Representa aproximadament les fronteres de decisió en 2D per als dos models en dues figures separades.

Compara els resultats: hi ha senyals de sobreajustament a l'arbre A? Explica el que observes.

In [ ]:
from sklearn.datasets import make_blobs

# Conjunt de dades 2D amb 2 centres molt separats i 500 observacions
X, y = make_blobs(
    n_samples=500,
    centers=[(-3, -3), (3, 3)],   # separem els centres
    n_features=2,
    cluster_std=2,                # Deixem així per tal de que no surti tot amb precisió 1     
    random_state=RANDOM_STATE
)

#Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_STATE
)

#Fem els dos arbres de decisió:
tree_A = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,
    random_state=RANDOM_STATE
)
tree_A.fit(X_train, y_train)

tree_B = DecisionTreeClassifier(
    criterion="gini",
    max_depth=3,
    random_state=RANDOM_STATE
)
tree_B.fit(X_train, y_train)

#Imprimim els resultats del accuracy en train i test
print("Arbre A")
print("Train accuracy:", tree_A.score(X_train, y_train))
print("Test accuracy :", tree_A.score(X_test, y_test))

print("\nArbre B")
print("Train accuracy:", tree_B.score(X_train, y_train))
print("Test accuracy :", tree_B.score(X_test, y_test))


In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
import matplotlib.pyplot as plt

#Fem les figures que representin les fronteres de decisió dels dos models usant la funció DecisionBoundaryDisplay
plt.figure(figsize=(6, 5))
DecisionBoundaryDisplay.from_estimator(
    tree_A,
    X_train,
    response_method="predict",
    cmap="bwr",
    alpha=0.3
)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap="bwr", edgecolor="k")
plt.title("Frontera de decisió de l'Arbre A ")
plt.show()

plt.figure(figsize=(6, 5))
DecisionBoundaryDisplay.from_estimator(
    tree_B,
    X_train,
    response_method="predict",
    cmap="bwr",
    alpha=0.3
)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap="bwr", edgecolor="k")
plt.title("Frontera de decisió de l'Arbre B")
plt.show()


En aquest exercici es demanava generar un conjunt de dades 2D de classificació binària, fer un train/test split i entrenar dos arbres de decisió amb profunditats diferents, després s'ha calculat l'accuracy.

Notem que per fer els centres ben separats (i així tenir els grups separats), en comptes de posar un valor molt baix en el cluster_std, s'han separat imposant que un estigui (-3,-3) i l'altre (3,3), i s'ha deixat el cluster_std en 2 per tal de tenir dades interessants.

Tot i així, han sortit accuracys molt semblants, en el cas de l'arbre A, el train accuracy ha estat d'1 i el test accuracy de 0.94, pel que sabem que és capaç de classificar perfectament totes les observacions del conjunt d'entrenament, i que té un rendiment una mica pitjor en el cas del conjunt test.

En la seva representació, veiem que hi ha diversos talls verticals i horitzontals que s'adapten als punts d'entrenament, per tant, la frontera és complexa.

L'arbre B, que tenia la profunditat limitada, ha obtingut un train accuracy una mica pitjor (tot i ser molt elevat també) de 0.98, però un test accuracy superior al de l'A i respecte a la seva representació, la frontera de decisió és molt més simple.

Llavors podem concloure que limitar la profunditat millora la generalització del model, i que en el cas de l'arbre A hi ha hagut overfitting, és per això que l'arbre B ha aconseguit un millor equilibri de rendiment.


### Exercici 3: Profunditat òptima en el dataset de breast cancer

1. Fes un train/test split del dataset de breast cancer (si no el tens, torna'l a carregar).
2. Per a valors de max_depth entre 1 i 10, entrena un DecisionTreeClassifier (criteri Gini, random_state=RANDOM_STATE).
3. Per a cada valor de max_depth, calcula l'accuracy en train i en test.
4. Representa en una mateixa figura l'accuracy train i test en funció de max_depth.
5. Tria una profunditat "òptima" segons el comportament de les corbes.
6. Entrena de nou un arbre amb aquesta profunditat òptima i mostra la confusion_matrix i l'accuracy en test

In [ ]:
from sklearn.datasets import load_breast_cancer

#Carreguem el dataset i fem un train/test split
cancer = load_breast_cancer(as_frame=True)
X_cancer = cancer.data
y_cancer = cancer.target

X_train, X_test, y_train, y_test = train_test_split(
    X_cancer, y_cancer,
    test_size=0.3,
    random_state=RANDOM_STATE,
    stratify=y_cancer
)

X_train.shape, X_test.shape

#Fem un bucle que entreni un DecisionTreeClassifier amb valors del max_depth del 1 al 10
max_depth_valors = range(1, 11)

train_accuracies = []
test_accuracies = []

for d in max_depth_valors:
    tree = DecisionTreeClassifier(
        criterion="gini",
        max_depth=d,
        random_state=RANDOM_STATE
    )
    tree.fit(X_train, y_train)
    
    train_acc = tree.score(X_train, y_train)
    test_acc  = tree.score(X_test, y_test)

    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    #L'accuracy de cada valor:
    print(f"Depth {d}: train={train_acc:.4f}, test={test_acc:.4f}")

#Representem graficament l'accuracy train i test en funció dels max_depth
plt.figure(figsize=(7, 4))
plt.plot(max_depth_valors, train_accuracies, marker="o", label="Train accuracy")
plt.plot(max_depth_valors, test_accuracies, marker="o", label="Test accuracy")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.title("Accuracy en train i test segons max_depth")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Busquem quin és el millor valor per trobar la profunditat òptima i el guardem
idx_best = int(np.argmax(test_accuracies))
prof_optima = list(max_depth_valors)[idx_best]

# Imprimim el resultat trobat i els seus accuracies corresponents
print("Profunditat òptima segons test accuracy:", prof_optima)
print("Train accuracy a la prof. òptima:", train_accuracies[idx_best])
print("Test accuracy a la prof. òptima :", test_accuracies[idx_best])


# Entrenem l'arbre definitiu amb la profunditat òptima
tree_cancer_opt = DecisionTreeClassifier(
    criterion="gini",
    max_depth=prof_optima,
    random_state=RANDOM_STATE
)
tree_cancer_opt.fit(X_train, y_train)

#Prediccions i accuracy al test
y_pred_opt = tree_cancer_opt.predict(X_test)
acc_opt = accuracy_score(y_test, y_pred_opt)

#Matriu de confusió
cm_optima = confusion_matrix(y_test, y_pred_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_optima, display_labels=cancer.target_names)
disp.plot(cmap="Blues")
plt.title(f"Breast cancer – Matriu de confusió (max_depth={prof_optima})")
plt.show()

# Informe de classificació
print("Classification report (test):")
print(classification_report(y_test, y_pred_opt, target_names=cancer.target_names))



S'ha fet un model d'entrenament del dataset Cancer Breast per estudiar la relació entre la profunditat màxima de l’arbre de decisió (max_depth) i la capacitat de generalització del model, és per això que s'ha fet un bucle que entreni un DecisionTree amb diferents profunditats amb criteri Gini, i s'ha calculat l'accuracy amb cadascun dels valors tant en train com en test, a la gràfica es pot veure els diferents valors dels dos tipus de dades.

Notem com l'accuracy en train augmenta al màxim a partir del max_depth 6, indicant que l'arbre memoritza les dades d'entrenament a la perfecció, però, en canvi, en l'accuracy del test creix al principi fins a obtenir el màxim al max_depth 5 i després disminueix lleugerament i es manté al 0.918, això pot ser per un overfitting, el model ha memoritzat tant el conjunt train que no sap generalitzar, i per això empitjora en augmentar profunditat.

Després, s'ha trobat la profunditat òptima, la qual ha estat amb max_depth 5, que és la que té major accuraccy test i s'ha fet el seu model d'entrenament i una matriu de confusió, així com les seves mètriques corresponents. Amb aquesta profunditat s'aconsegueix un train accuracy del 0.995 i un test d'accuracy de 0.93. La matriu de confusió ens deixa veure que es classifiquen correctament 58 tumors malignes i 101 benignes, i 6 malignes són classificats com a benignes, i 6 benignes com a malignes. Per tant, amb tota aquesta informació veiem un bon rendiment global tot i que es pot millorar.

En conseqüència, acabem de veure que a vegades limitar la profunditat de l'arbre a valors intermedis és una bona opció que permet deixar el model equilibrat amb capacitat de generalització.

### Exercici 4: Comparació de criteris de divisió: Gini vs Entropia

1. Utilitza el mateix train/test split de l'Exercici 3.
2. Entrena un arbre amb:
- Model G: DecisionTreeClassifier(criterion="gini", max_depth=best_depth, random_state=RANDOM_STATE)
- Model E: DecisionTreeClassifier(criterion="entropy", max_depth=best_depth, random_state=RANDOM_STATE) (pots reutilitzar best_depth de l'Exercici 3 o triar un valor raonable, per exemple max_depth=4).
3. Calcula l'accuracy en train i en test per a cada model.
4. Mostra la confusion_matrix dels dos models.
5. Comenta si hi ha diferències importants entre Gini i Entropia en aquest cas.

In [ ]:
# Profunditat òptima trobada abans
best_depth = 5

# Model G (criteri Gini)
tree_g = DecisionTreeClassifier(
    criterion="gini",
    max_depth=best_depth,
    random_state=RANDOM_STATE
)
tree_g.fit(X_train, y_train)

# Model E (criteri Entropia)
tree_e = DecisionTreeClassifier(
    criterion="entropy",
    max_depth=best_depth,
    random_state=RANDOM_STATE
)
tree_e.fit(X_train, y_train)

print("MODEL G")
print("Train accuracy:", tree_g.score(X_train, y_train))
print("Test accuracy :", tree_g.score(X_test, y_test))

print("\nMODEL E")
print("Train accuracy:", tree_e.score(X_train, y_train))
print("Test accuracy :", tree_e.score(X_test, y_test))

# Prediccions
y_pred_g = tree_g.predict(X_test)
y_pred_e = tree_e.predict(X_test)

# Matriu de confussió del model G
cm_g = confusion_matrix(y_test, y_pred_g)
disp_g = ConfusionMatrixDisplay(confusion_matrix=cm_g, display_labels=cancer.target_names)
disp_g.plot(cmap="Blues")
plt.title(f"Matriu de confusió del Model G (Gini, depth={best_depth})")
plt.show()

# Ara la del model E
cm_e = confusion_matrix(y_test, y_pred_e)
disp_e = ConfusionMatrixDisplay(confusion_matrix=cm_e, display_labels=cancer.target_names)
disp_e.plot(cmap="Blues")
plt.title(f"Matriu de confusió del Model E (Entropia, depth={best_depth})")
plt.show()



Ara, amb el mateix train/test split d'abans en el que havíem trobat que la millor profunditat era 5, s'han entrenat dos arbres, un amb el criteri Gini i l'altre amb l'Entropia, s'ha calculat l'accuracy en train i test, i s'han obtingut les matrius de confusions de cada model.

Els resultats mostren que tenen un comportament molt similar. L'accuracy en train és aproximadament de 0.995 en els dos models, però en accuracy en test sí que canvia una mica, en el cas del criteri Gini, tenim un accuracy de 0.93 i en Entropia de 0.95, veient així que el criteri Entropia té una millor capacitat de generalització.

Amb les matrius de confusió reforcem aquesta idea, en el Model E s'han obtingut 8 errors en total, mentre que en el model G s'han trobat 12 errors, per tant, el model E és lleugerament millor, ja que redueix els falsos positius i falsos negatius. Tot i així tenim un bon rendiment i molt similar en ambdós casos.

Com a resultat, amb aquesta profunditat i aquest dataset, podem dir que amb els dos criteris de divisió, obtenim molt bons resultats i un bon rendiment, però que el criteri d'Entropia dona millor capacitat de generalització, per tant és lleugerament millor.